In [2]:
import torch
import numpy as np
from fastai.text.all import *

In [3]:
path = untar_data(URLs.MNIST_SAMPLE)

In [5]:
path.ls()

(#3) [Path('/Users/fabianjaeger/.fastai/data/mnist_sample/valid'),Path('/Users/fabianjaeger/.fastai/data/mnist_sample/labels.csv'),Path('/Users/fabianjaeger/.fastai/data/mnist_sample/train')]

### Building Basic RNN

In [5]:
import torch
import torch.nn as nn
torch.manual_seed(1)

### Sentiment Analysis on IMDb Movie Reviews

Corresponds to a many-to-one RNN problem (i.e sentence or tokens get classified by a fixed label, `neg` or `pos` encoded as `1`, `2` respectively)


1. Split data into training, validation and test set
2. Find the unique words in the training set
3. Encode each unique word by a mapping to a unique integer and actually encode the review text of the movies into encoded integers
4. Divide the dataset into mini-batches as input to the model

In [6]:
from torchtext.datasets import IMDB
train_iter = IMDB(split='train') # Whole dataset contains 25'000 movie reviews
test_iter = IMDB(split="test") # contains 15'000 movie reviews

from torch.utils.data import DataLoader, Dataset

ModuleNotFoundError: No module named 'torchtext'

Split data into training, validation and test set

In [4]:
from torch.utils.data.dataset import random_split
torch.manual_seed(1)
train_dataset, valid_dataset = random_split(list(train_iter), [20000, 5000])

In [17]:
list(train_dataset[400])

[1,
 "I normally have no problem walking away from a bad movie, however this was an unique case. This movie was so bad that I actually sat through the whole thing almost praying it would have one minute of good movie time to justify the hour and a half that was wasted. Needless to say I was brutally disappointed. Set at a beach house where a group of college friends are celebrating vacation, this movie suffers from numerous problems making it not worth seeing. First, there are gaping plot holes. Second, very few of the C-list (i don't even dare call them B) actors can act worth a damn, so any scenes that have potential fail miserably. Third, the rate of the film is very choppy and awkward to watch most of the time making suspense building very difficult, leading to very few surprises for the audience. Fourth and most importantly, the ending is completely anti-climatic partially because of how it ends (setting/who the killer turns out to be) and partially because the dialog is just atro

In [78]:
import pandas as pd

def get_df(data_iter):
    labels = []
    reviews = []
    for label, review in data_iter:
        labels.append(label)
        reviews.append(review) 
    return labels,reviews
    
train_label, train_reviews = get_df(train_dataset)
val_label, val_reviews = get_df(valid_dataset)
test_label, test_reviews = get_df(test_iter)

test_df = pd.DataFrame({"sentiment": test_label, "reviews": test_reviews})
val_df= pd.DataFrame({"sentiment": val_label, "reviews": val_reviews})
train_df = pd.DataFrame({"sentiment": train_label, "review": train_reviews})


In [1]:
import re
from collections import Counter, OrderedDict

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text) 
    emoticons = re.findall(
        '(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower()
    )
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    tokenized = text.split()
    return tokenized

token_counts = Counter() 

for label, line in train_dataset:
    tokens = tokenizer(line)
    token_counts.update(tokens)

print('Vocab-size:', len(token_counts))

NameError: name 'train_dataset' is not defined

In [79]:
# display(train_df[train_df["sentiment"] == 2])
display(train_df)

,sentiment,review
0,2,"An extra is called upon to play a general in a movie about the Russian Revolution. However, he is not any ordinary extra. He is Serguis Alexander, former commanding general of the Russia armies who is now being forced to relive the same scene, which he suffered professional and personal tragedy in, to satisfy the director who was once a revolutionist in Russia and was humiliated by Alexander. It can now be the time for this broken man to finally ""win"" his penultimate battle. This is one powerful movie with meticulous direction by Von Sternberg, providing the greatest irony in Alexander's c..."
1,2,"almost every review of this movie I'd seen was pretty bad. It's not pretty bad, it's actually pretty good, though not great. The Judy Garland character could have gotten annoying, but she didn't allow it to. Somewhere along the line, i've become a fan of brooding, overbearing, overacting Van Heflin, at least in the early 40's. Judy's singing is great, but the film missed a great chance by not showing more of their relationship. I gave it a 7."
2,2,"I did not have too much interest in watching The Flock.Andrew Lau co-directed the masterpiece trilogy of Infernal Affairs but he had been fired from The Flock and he had been replaced by an emergency director called Niels Mueller.I had the feeling that Lau had made a good film but it had not satisfied the study,so they fired him and hired another director.This usually does not work well (let's remember The Invasion).But The Flock resulted to be better than what I expected.It's not a great film but it's an interesting and entertaining thriller.The character development is very well done and..."
3,1,"Ulises is a literature teacher that arrives to a coastal town. There, he will fell in love to Martina, the most beautiful girl in town. They will start a torrid romance which will end in the tragic death of Ulises at the sea. Some years later, Martina has married to Sierra, the richest man in town and lives a quiet happy live surrounded by money. One day, the apparition of Ulises will make her passion to rise up and act without thinking the consequences. The plot is quite absurd and none of the actors plays a decent part. IN addition, three quarters of the film are sexual acts, which, stil..."
4,2,"I found The FBI Story considerably entertaining and suitably upbeat for my New Years Day holiday viewing. Its drama and action-packed episodes were thrilling. The Hardesty character was well drawn and admirable. Overall the photography, script and direction was perfectly creditable. Rather than taking the film to be a repugnant piece of propaganda, as some might, I enjoyed it as a well mounted portrayal of the necessity of ingenious minds and brave bodies in the fight against crime. Again, the depiction of a family holding together even under the strain of the husband's commitment to his (..."
...,...,...
19995,1,"Let me begin by saying I am a big fantasy fan. However, this film is not for me. Many far-fetched arguments are trying to support this film's claim that dragons possibly ever existed. The film mentions connections in different stories from different countries, but fails to investigate them more thoroughly, which could have given the film some credibility. The film uses (nice!) CGI to tell us a narrated fantasy story on a young dragon's life. This is combined with popular-TV-show-CSI-style flash-forwards to make it look like something scientific, which it is definitely not. In many cases th..."
19996,1,"Well, I just ordered this on my pay-per-view at home because I was bored and needed a laugh. I have to admit, I did chuckle a few times, but I don't even remember what parts they were at. I don't understand why this movie was made. It claims to be a comedy but seriousuly, I don't find a singing penis, or a naked 70 year old woman very funny. This movie was trying to fit itself into the 'gross-out' comedies of recent years such as American Pie and Road Trip, but it jus

Convert categorial integers `1` and `2` into labels `neg` and `pos` respectively

In [84]:
def apply_df_transform(df):
    df["sentiment"] = df['sentiment'].apply(lambda x: "neg" if x == 1 else ("pos" if x == 2 else x))

for df in [test_df, train_df, val_df]:
    apply_df_transform(df)
    
display(train_df)

,sentiment,review
0,pos,"An extra is called upon to play a general in a movie about the Russian Revolution. However, he is not any ordinary extra. He is Serguis Alexander, former commanding general of the Russia armies who is now being forced to relive the same scene, which he suffered professional and personal tragedy in, to satisfy the director who was once a revolutionist in Russia and was humiliated by Alexander. It can now be the time for this broken man to finally ""win"" his penultimate battle. This is one powerful movie with meticulous direction by Von Sternberg, providing the greatest irony in Alexander's c..."
1,pos,"almost every review of this movie I'd seen was pretty bad. It's not pretty bad, it's actually pretty good, though not great. The Judy Garland character could have gotten annoying, but she didn't allow it to. Somewhere along the line, i've become a fan of brooding, overbearing, overacting Van Heflin, at least in the early 40's. Judy's singing is great, but the film missed a great chance by not showing more of their relationship. I gave it a 7."
2,pos,"I did not have too much interest in watching The Flock.Andrew Lau co-directed the masterpiece trilogy of Infernal Affairs but he had been fired from The Flock and he had been replaced by an emergency director called Niels Mueller.I had the feeling that Lau had made a good film but it had not satisfied the study,so they fired him and hired another director.This usually does not work well (let's remember The Invasion).But The Flock resulted to be better than what I expected.It's not a great film but it's an interesting and entertaining thriller.The character development is very well done and..."
3,neg,"Ulises is a literature teacher that arrives to a coastal town. There, he will fell in love to Martina, the most beautiful girl in town. They will start a torrid romance which will end in the tragic death of Ulises at the sea. Some years later, Martina has married to Sierra, the richest man in town and lives a quiet happy live surrounded by money. One day, the apparition of Ulises will make her passion to rise up and act without thinking the consequences. The plot is quite absurd and none of the actors plays a decent part. IN addition, three quarters of the film are sexual acts, which, stil..."
4,pos,"I found The FBI Story considerably entertaining and suitably upbeat for my New Years Day holiday viewing. Its drama and action-packed episodes were thrilling. The Hardesty character was well drawn and admirable. Overall the photography, script and direction was perfectly creditable. Rather than taking the film to be a repugnant piece of propaganda, as some might, I enjoyed it as a well mounted portrayal of the necessity of ingenious minds and brave bodies in the fight against crime. Again, the depiction of a family holding together even under the strain of the husband's commitment to his (..."
...,...,...
19995,neg,"Let me begin by saying I am a big fantasy fan. However, this film is not for me. Many far-fetched arguments are trying to support this film's claim that dragons possibly ever existed. The film mentions connections in different stories from different countries, but fails to investigate them more thoroughly, which could have given the film some credibility. The film uses (nice!) CGI to tell us a narrated fantasy story on a young dragon's life. This is combined with popular-TV-show-CSI-style flash-forwards to make it look like something scientific, which it is definitely not. In many cases th..."
19996,neg,"Well, I just ordered this on my pay-per-view at home because I was bored and needed a laugh. I have to admit, I did chuckle a few times, but I don't even remember what parts they were at. I don't understand why this movie was made. It claims to be a comedy but seriousuly, I don't find a singing penis, or a naked 70 year old woman very funny. This movie was trying to fit itself into the 'gross-out' comedies of recent years such as American Pie and Road Tr

Encoding unique tokens (~words) to numeric values

In [8]:
from fastai.text.all import *

In [11]:
tokenizer = WordTokenizer()
toks = first(tokenizer([list(train_dataset)]))
toks

ValueError: [E088] Text of length 26603675 exceeds maximum of 1000000. The parser and NER models require roughly 1GB of temporary memory per 100,000 characters in the input. This means long texts may cause memory allocation errors. If you're not using the parser or NER, it's probably safe to increase the `nlp.max_length` limit. The limit is in number of characters, so you can check whether your inputs are too long by checking `len(text)`.